# Osprey Report
---

### Imports and convenience functions

In [ ]:
get_ipython().magic('pylab inline')
matplotlib.rcParams['figure.dpi'] = 600

import seaborn as sns
import corner

import json
from StringIO import StringIO

import numpy as np
from pandas import DataFrame, Index
from scipy.stats import gaussian_kde

from osprey.config import Config
from msmbuilder.msm import BayesianMarkovStateModel

from msmreport.utils import convert_keys_to_string
from msmreport.plotting import plot_bars, plot_free_energy

### Retrieve osprey settings

In [ ]:
config = Config({yaml})
ss = config.search_space()

### Retrieve optimal model parameters

In [ ]:
db = config.trials()
cmd = 'select parameters from trials_v3 where mean_test_score = (select MAX(mean_test_score) from trials_v3);'
query = db.execute(cmd)
result = query.fetchall()
params = convert_keys_to_string(json.load(StringIO(result[0][0])))
DataFrame(np.array(params.values())[argsort(params.keys())], Index(sort(params.keys()))).T

### Load data and model pipeline

In [ ]:
data, labels = config.dataset()

pipeline = config.estimator()
pipeline.set_params(**params)

### Fit model

In [ ]:
pipeline.fit(data)

### tICA results

In [ ]:
tica = pipeline.steps[0][1].transform(data)
_ = corner.corner(np.vstack(tica)[::100, :])

### Clustering results

In [ ]:
scatter(*np.vstack(tica)[::100, :2].T, alpha=.3, color='gray')
s = np.bincount(np.hstack(pipeline.steps[1][1].labels_))
scatter(*pipeline.steps[1][1].cluster_centers_[:, :2].T,
        s=1E3*s/s.sum(), alpha=.3, color='red')

### MSM results

**Plot top five timescales**

In [ ]:
lt_range = ss.variables['tica__lag_time']
lag_times = np.linspace(lt_range.min, lt_range.max, 20).astype(int)

models = []
timescales = []
for lag_time in lag_times:
    model = BayesianMarkovStateModel(verbose=False, n_timescales=5,
                                     lag_time=lag_time, ergodic_cutoff=1)
    models.append(model.fit(pipeline.steps[1][1].labels_))
    timescales.append(model.all_timescales_)
timescales = np.array(timescales)

figsize(12, 6)
for timescale in timescales.T:
    _ = fill_between(lag_times, (np.mean(timescale, 0) - 2 * np.std(timescale, 0)),
                     (np.mean(timescale, 0) + 2 * np.std(timescale, 0)),
                     color='gray', alpha=0.2)
    _ = plot(lag_times, np.mean(timescale, 0))

xlabel('Lag Time\n($ns$)', size=14, labelpad=10)
ylabel('Relaxation Time ($ns$)', size=14, labelpad=10)
yscale('log')

**Plot "best" timescale**

In [ ]:
ts = np.mean(timescales, 1) - 2 * np.std(timescales, 1)
tidx = argmax(np.product((np.diff(ts, axis=0) / ts[:-1]) < 0.01, axis=1))
lag_time = lag_times[tidx]
msm = models[tidx]

figsize(2,8)

fig, ax = plot_bars(timescales[tidx].mean(0), timescales[tidx].std(0), upper=1E5)
ax.set_title('Lag Time: %s $ns$' % lag_time, size=16, y=1.05)

**Plot residuals**

In [ ]:
sns.jointplot(s[msm.mapping_.keys()]/float(s[msm.mapping_.keys()].sum()),
              msm.all_populations_.mean(0), kind='resid')
xlabel('Raw Populations', size=16)
ylabel('Residuals', size=16)
xlim([1E-4, 1])
ylim([-1E-2, 1E-2])
xscale('log')
yscale('symlog')

**Plot free energy landscape**

In [ ]:
p = msm.all_populations_.mean(0)[np.hstack(msm.transform(pipeline.steps[1][1].labels_))]
p /= p.sum()
fig, ax = plot_free_energy(np.vstack(tica), p=p)